In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from uuid import uuid4
import os
load_dotenv()

True

In [6]:
pdf_loader = PyPDFLoader("../dataset/UAS.pdf")
pages = []
async for page in pdf_loader.alazy_load():
    pages.append(page)

In [7]:
print(f"Terdapat {len(pages)} Halaman")
pages

Terdapat 55 Halaman


[Document(metadata={'source': '../dataset/UAS.pdf', 'page': 0}, page_content='PROYEK SISTEM TERDISTRIBUSI \nIMPLEMENTASI SISTEM PENJUALAN PISANG KRISPY \nDENGAN SISTEM TERDISTRIBUSI \n \n \n \nDisusun Oleh : \nSaiyidati Vienna Arum Pratama      200411100018 \nAhmad Rosyihuddin                     200411100126 \nSiti Ulun Nuha Karimah                  200411100127 \nAhmad Fanani                              200411100143 \n \n \n \nPROGRAM STUDI TEKNIK INFORMATIKA \nJURUSAN TEKNIK INFORMATIKA \nFAKULTAS TEKNIK \nUNIVERSITAS TRUNOJOYO MADURA \n2022\n'),
 Document(metadata={'source': '../dataset/UAS.pdf', 'page': 1}, page_content='i | Proyek Sistem Terdistribusi Kelompok 4 (Copyright 2022) \n \nDAFTAR ISI \nDAFTAR ISI ................................ ................................ ................................ .................  i \nBAB I PENDAHULUAN ................................ ................................ .............................  1 \n1.1. Latar Belakang ...............

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Ukuran potongan dalam karakter
    chunk_overlap=200  # Jumlah karakter yang tumpang tindih antar potongan
)

In [32]:
documents = []
for page in pages:
    split_doc = text_splitter.split_text(page.page_content)
    uuid = str(uuid4())
    for content in split_doc:
        documents.append(Document(metadata={"parent_id": uuid, "source": page.metadata["source"], "page": page.metadata["page"]}, page_content=content))

In [33]:
uuids = [str(uuid4()) for _ in documents]

In [34]:
print(f"Banyak Chunk dari {len(pages)} Halaman: {len(documents)}")
documents

Banyak Chunk dari 55 Halaman: 82


[Document(metadata={'parent_id': 'e9ae162b-06a6-4b64-a641-8a83c5244cba', 'source': '../dataset/UAS.pdf', 'page': 0}, page_content='PROYEK SISTEM TERDISTRIBUSI \nIMPLEMENTASI SISTEM PENJUALAN PISANG KRISPY \nDENGAN SISTEM TERDISTRIBUSI \n \n \n \nDisusun Oleh : \nSaiyidati Vienna Arum Pratama      200411100018 \nAhmad Rosyihuddin                     200411100126 \nSiti Ulun Nuha Karimah                  200411100127 \nAhmad Fanani                              200411100143 \n \n \n \nPROGRAM STUDI TEKNIK INFORMATIKA \nJURUSAN TEKNIK INFORMATIKA \nFAKULTAS TEKNIK \nUNIVERSITAS TRUNOJOYO MADURA \n2022'),
 Document(metadata={'parent_id': '039e31a1-4904-4817-a706-eaca94c3776f', 'source': '../dataset/UAS.pdf', 'page': 1}, page_content='i | Proyek Sistem Terdistribusi Kelompok 4 (Copyright 2022) \n \nDAFTAR ISI \nDAFTAR ISI ................................ ................................ ................................ .................  i \nBAB I PENDAHULUAN ................................

In [3]:
embeddings =  HuggingFaceEmbeddings(
        model_name="LazarusNLP/all-indo-e5-small-v4",     # Provide the pre-trained model's path
        model_kwargs={'device':'cpu'}, # Pass the model configuration options
        encode_kwargs={'normalize_embeddings': False}, # Pass the encoding options
        show_progress=True
    )

/home/pstar7/Documents/chatbot/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
db = FAISS.from_documents(documents=documents, embedding=embeddings, ids=uuids)

Batches: 100%|██████████| 3/3 [00:04<00:00,  1.58s/it]


In [37]:
db.docstore._dict

{'c26e3230-e0e9-4c72-bef6-651385a3acd0': Document(id='c26e3230-e0e9-4c72-bef6-651385a3acd0', metadata={'parent_id': 'e9ae162b-06a6-4b64-a641-8a83c5244cba', 'source': '../dataset/UAS.pdf', 'page': 0}, page_content='PROYEK SISTEM TERDISTRIBUSI \nIMPLEMENTASI SISTEM PENJUALAN PISANG KRISPY \nDENGAN SISTEM TERDISTRIBUSI \n \n \n \nDisusun Oleh : \nSaiyidati Vienna Arum Pratama      200411100018 \nAhmad Rosyihuddin                     200411100126 \nSiti Ulun Nuha Karimah                  200411100127 \nAhmad Fanani                              200411100143 \n \n \n \nPROGRAM STUDI TEKNIK INFORMATIKA \nJURUSAN TEKNIK INFORMATIKA \nFAKULTAS TEKNIK \nUNIVERSITAS TRUNOJOYO MADURA \n2022'),
 '6856183f-4f46-44cb-b716-96ac0e74b936': Document(id='6856183f-4f46-44cb-b716-96ac0e74b936', metadata={'parent_id': '039e31a1-4904-4817-a706-eaca94c3776f', 'source': '../dataset/UAS.pdf', 'page': 1}, page_content='i | Proyek Sistem Terdistribusi Kelompok 4 (Copyright 2022) \n \nDAFTAR ISI \nDAFTAR ISI ......

In [38]:
question = "bagaimana caranya mereplikasi database"
resultE5Indo = db.similarity_search(question, k=10)
resultE5Indo

Batches: 100%|██████████| 1/1 [00:00<00:00, 36.77it/s]


[Document(id='60a8ee34-4dc5-4333-a0ee-695f386dfa39', metadata={'parent_id': '23e9be05-09b2-4387-b39f-d86469fcb9bc', 'source': '../dataset/UAS.pdf', 'page': 8}, page_content='agar instance-instance yang mempunyai IP Address Internal akan tetap dapat diakses oleh \ncontroller. Jadi nantinya, instance -instance terseb ut akan mempunyai dua buah IP \nAddress. IP Address pertama adalah IP Address Internal, dan yang kedua adalah external \nhasil dari floating IP. \n2.5. Replikasi Database \nReplikasi Database adalah suatu proses menyalin database ke database lain yang \nada di server yang sama atau server yang berbeda, dan mensinkronisasi kan datanya untuk \nmenjaga konsistensi. Dengan menggunakan Replikasi ini, data dapat didistribusikan ke \nlokasi yang berbeda dengan melalui jalur local yaitu LAN, WAN, atau internet. Replikasi'),
 Document(id='9ffd73b5-a14a-4ec1-be05-c4b3b0233748', metadata={'parent_id': 'd1b6cad6-18aa-49f1-80d7-2546f3b7eacb', 'source': '../dataset/UAS.pdf', 'page': 6}, p

In [39]:
db.similarity_search(query=question, k=10, filter={"parent_id": {"$eq": "402151c0-b1c3-434a-8de1-f1c5abfe8b84"}})

Batches: 100%|██████████| 1/1 [00:00<00:00, 41.10it/s]


[]

In [40]:
db.save_local("db_laporan")